In [1]:
import QuantLib as ql

In [2]:
today = ql.Date(24, 12, 2016)
ql.Settings.instance().evaluationDate = today

In [3]:
u = ql.SimpleQuote(100.0)
r = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.20)

risk_free_curve = ql.FlatForward(today, ql.QuoteHandle(r), ql.Actual360())
volatility = ql.BlackConstantVol(today, ql.TARGET(), ql.QuoteHandle(sigma), ql.Actual360())

In [4]:
process_1 = ql.BlackScholesProcess(ql.QuoteHandle(u), ql.YieldTermStructureHandle(risk_free_curve), ql.BlackVolTermStructureHandle(volatility))
print(process_1.dividendYield().zeroRate(1.0, ql.Continuous))

0.000000 % Actual/365 (Fixed) continuous compounding


In [5]:
process_2 = ql.BlackProcess(ql.QuoteHandle(u), ql.YieldTermStructureHandle(risk_free_curve), ql.BlackVolTermStructureHandle(volatility))
print(process_2.riskFreeRate().zeroRate(1.0, ql.Continuous))
print(process_2.dividendYield().zeroRate(1.0, ql.Continuous))

1.000000 % Actual/360 continuous compounding
1.000000 % Actual/360 continuous compounding


In [6]:
option_1 = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, 100.0), ql.EuropeanExercise(today+100))
option_1.setPricingEngine(ql.AnalyticEuropeanEngine(process_1))
print(option_1.NPV())

4.337597216336533


In [7]:
option_2 = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, 100.0), ql.EuropeanExercise(today+100))
option_2.setPricingEngine(ql.AnalyticEuropeanEngine(process_2))
print(option_2.NPV())

4.191615257389808


In [8]:
def greek(option, quote, dx):
    x0 = quote.value()
    quote.setValue(x0+dx)
    P_u = option.NPV()
    quote.setValue(x0-dx)
    P_d = option.NPV()
    quote.setValue(x0)
    return (P_u-P_d)/(2*dx)

In [9]:
print(option_1.delta())
print(greek(option_1, u, 0.01))

0.5315063340142601
0.531506323010289


In [10]:
print(option_2.delta())
print(greek(option_2, u, 0.01))

0.5195711146255227
0.5195711052036867


In [12]:
print(option_1.vega())
print(greek(option_1, sigma, 0.001))

20.96050033373808
20.960499909565833


In [13]:
print(option_2.vega())
print(greek(option_2, sigma, 0.001))

20.938677847075486
20.938677605407463


In [14]:
print(option_1.rho())
print(greek(option_1, r, 0.001))

13.559176718080407
13.55917453385036


In [15]:
print(option_2.rho())
print(greek(option_2, r, 0.001))

13.268193390322908
-1.1643375864700545


In [16]:
print(option_2.rho() + option_2.dividendRho())
print(greek(option_2, r, 0.001))

-1.1643375714971693
-1.1643375864700545


In [17]:
process_3 = ql.BlackScholesMertonProcess(ql.QuoteHandle(u), ql.YieldTermStructureHandle(risk_free_curve), ql.YieldTermStructureHandle(risk_free_curve), ql.BlackVolTermStructureHandle(volatility))
option_3 = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, 100.0), ql.EuropeanExercise(today+100))
option_3.setPricingEngine(ql.AnalyticEuropeanEngine(process_3))

In [18]:
print(option_3.delta())
print(greek(option_3, u, 0.01))

0.5195711146255227
0.5195711052036867


In [19]:
print(option_3.rho())
print(option_3.rho() + option_3.dividendRho())
print(greek(option_3, r, 0.001))

13.268193390322908
-1.1643375714971693
-1.1643375864700545
